In [60]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import pickle
from pickle import dump
from sklearn.metrics import mean_squared_error, r2_score
import math

In [61]:
y_train = pd.read_csv('/workspaces/EDA_4/data/processed/heart_prevalence_y_train.csv')
y_test = pd.read_csv('/workspaces/EDA_4/data/processed/heart_prevalence_y_test.csv')
X_train_norm = pd.read_csv('/workspaces/EDA_4/data/interim/heart_prevalence_X_train_std.csv')
X_test_norm = pd.read_csv('/workspaces/EDA_4/data/interim/heart_prevalence_X_test_std.csv')

In [62]:
from sklearn.feature_selection import f_regression, SelectKBest

In [63]:
rmss = []
r2s = []
percents = [1, 0.8, 0.7, 0.6, 0.5]
for p in percents:

    selection_model = SelectKBest(f_regression, k = int(len(X_train_norm.columns) * p))
    selection_model.fit(X_train_norm, y_train)
    ix = selection_model.get_support()

    X_train_sel = pd.DataFrame(selection_model.transform(X_train_norm), columns = X_train_norm.columns.values[ix])
    X_test_sel = pd.DataFrame(selection_model.transform(X_test_norm), columns = X_test_norm.columns.values[ix])

    dump(selection_model, open(f"/workspaces/EDA_4/models/selection_model{p}.pk", "wb"))

    model = LinearRegression()
    model.fit(X_train_sel, y_train)
    y_pred = model.predict(X_train_sel)
    rmss.append(math.sqrt(mean_squared_error(y_train, y_pred)))
    r2s.append(r2_score(y_train, y_pred))

best_rmss = rmss.index(max(rmss)) 
best_r2s = r2s.index(max(r2s)) 

print(f"Error cuadrático medio: {rmss[best_rmss]}")
print(f"Coeficiente de determinación: {r2s[best_r2s]}")
print(f"Best model is {percents[best_rmss]}")

Error cuadrático medio: 0.2794007435544796
Coeficiente de determinación: 0.9579028289114827
Best model is 0.5


/home/vscode/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/vscode/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/vscode/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/vscode/.local/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

In [64]:
selection_model = pickle.load(open("/workspaces/EDA_4/models/selection_model0.5.pk", "rb"))
ix = selection_model.get_support()

X_train_sel = pd.DataFrame(selection_model.transform(X_train_norm), columns = X_train_norm.columns.values[ix])
X_test_sel = pd.DataFrame(selection_model.transform(X_test_norm), columns = X_test_norm.columns.values[ix])

model = LinearRegression()
model.fit(X_train_sel, y_train)

LinearRegression()

In [65]:
print(f"Intercept (a): {model.intercept_}")
print(f"Coefficients (b): {model.coef_}")

Intercept (a): [8.37149865]
Coefficients (b): [[-0.15549461  0.75459943 -0.27330536  0.53737403 -0.61739438  0.56593799]]


In [66]:
y_pred = model.predict(X_train_sel)
print(f"Error cuadrático medio: {math.sqrt(mean_squared_error(y_train, y_pred))}")
print(f"Coeficiente de determinación: {r2_score(y_train, y_pred)}")

Error cuadrático medio: 0.2794007435544796
Coeficiente de determinación: 0.9340762739842043


In [67]:
y_pred = model.predict(X_test_sel)
print(f"Error cuadrático medio: {math.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"Coeficiente de determinación: {r2_score(y_test, y_pred)}")

Error cuadrático medio: 0.5255850187397291
Coeficiente de determinación: 0.8686706799451416
